# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Jaidev Mirchandani
- Bora Mutluoglu
- Yuqi Zhang
- Keven Wilbert Saputra
- Weiqi Peng

# Group Members IDs

- A92092333
- A13458532
- A15155513
- A14772056
- A14416515

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [3]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [4]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

The dataset we have chosen is publicly available to us. Hence, there has not been a need to request permission for access to the data. The website we have taken the data from, namely Kaggle, is a website that focuses on data privacy. In order to download datasets, we sign agreements on various aspects of data privacy. It is made clear to us that the data is anonymized and/or based on a random sampling of the population. 

Moreover, all the data samples are anonymous and do not involve any sensitive information that might cause trouble to the providers of those data. We analyze the data only for study reasons and for this course project. The data we have downloaded from Kaggle will not used by this group for any financial gains or business purposes. 

There exist potential biases in the observations put forth in the dataset. Potential biases may include families who come from wealthy areas in California such as many places in the Silicon Valley or well off areas in the Los Angeles/San Diego counties. These families that come from these areas may be used to abnormally high house/apartment prices due to potential safety/better quality of life from living in these communities. Schools also play a large part of housing prices since areas with renowned schools generally boast much higher house prices compared to other places that might give out more land for less of a price. However, there might be a risk of missing out on residents since the collection of data is done on a block group basis. The definition of a block group is given in the “Data” section of the proposal. 


# Conclusion & Discussion

*Fill in your discussion information here*